In [1]:
# Working example
import pandas as pd
from cytominer_eval import evaluate

# Load Data
path = "profiles/NCP_PILOT_3/BR_NCP_PILOT_3/BR_NCP_PILOT_3_normalized_variable_selected.csv.gz"

df = pd.read_csv(path)

# Define important function arguments
meta_features = df.columns[df.columns.str.startswith("Metadata_")]
features = df.drop(meta_features, axis="columns").columns.tolist()


In [2]:
# Metadata_group is Metadata_line_condition + Metadata_plating_density
# Metadata_replicate is Metadata_compound_ID

df.head(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Metadata_group,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
0,BR_NCP_PILOT_3,A01,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A01,2500,control,Untreated,Untreated,control_2500,...,0.68736,0.32353,0.38678,-1.333400,0.84262,0.43418,2.85160,-0.065548,1.05590,1.8216
1,BR_NCP_PILOT_3,A02,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A02,10000,control,Untreated,Untreated,control_10000,...,1.42290,-1.49340,-1.79440,-0.053887,1.81270,2.02500,4.02830,-0.377890,1.38140,2.0071
2,BR_NCP_PILOT_3,A03,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A03,2500,control,BRD-K79131256-001-17-9,BRD-K79131256-001-17-9,control_2500,...,0.83461,-0.84041,-0.30974,0.854620,1.29650,0.03451,0.90154,-0.094023,0.67013,2.7642
3,BR_NCP_PILOT_3,A04,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A04,10000,control,BRD-K36207157-001-09-6,BRD-K36207157-001-09-6,control_10000,...,1.66050,-1.73520,-1.53300,-0.472280,1.98300,1.34460,1.43450,0.477740,1.00020,2.0469
4,BR_NCP_PILOT_3,A05,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A05,2500,control,Untreated,Untreated,control_2500,...,2.11990,1.42280,1.51170,0.554710,-0.27509,-0.17327,0.92286,-0.267170,0.50922,1.2480


In [3]:
df.sample(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Metadata_group,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
16,BR_NCP_PILOT_3,A17,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A17,2500,control,Untreated,Untreated,control_2500,...,0.82166,0.14628,0.82596,0.19543,0.15887,-0.333040,0.552840,-0.910150,-0.000374,1.14930
307,BR_NCP_PILOT_3,M20,BR_NCP_PILOT_3,BR_NCP_PILOT_3,M20,10000,control,Untreated,Untreated,control_10000,...,-0.37452,-1.31460,-0.87439,-0.35005,1.36820,-0.127470,-0.528830,0.127360,-0.119760,0.15464
113,BR_NCP_PILOT_3,E18,BR_NCP_PILOT_3,BR_NCP_PILOT_3,E18,10000,control,BRD-K21680192-300-14-4,BRD-K21680192-300-14-4,control_10000,...,-0.76527,0.46927,-0.91781,0.22031,-1.22550,-1.323200,0.377350,-1.051200,-1.096900,-1.16080
298,BR_NCP_PILOT_3,M11,BR_NCP_PILOT_3,BR_NCP_PILOT_3,M11,2500,control,BRD-K21680192-300-14-4,BRD-K21680192-300-14-4,control_2500,...,-1.96600,-2.39840,-2.55160,0.43260,0.86430,-1.611100,-0.685350,-1.234800,-1.090800,0.11174
54,BR_NCP_PILOT_3,C07,BR_NCP_PILOT_3,BR_NCP_PILOT_3,C07,2500,deletion,Untreated,Untreated,deletion_2500,...,-0.46287,0.45597,0.54225,0.19196,-0.29236,0.048668,-0.071492,-0.004089,-0.266560,-0.28393


In [4]:
# Evaluate profile quality using grit
# First do this without any grouping (use the whole plate)

control_perts = [
    "Untreated"
]

replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_Plate",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

,perturbation,group,grit
0,BRD-K15108141-003-04-7,BR_NCP_PILOT_3,0.039541
1,BRD-K21680192-300-14-4,BR_NCP_PILOT_3,0.029373
2,BRD-K36207157-001-09-6,BR_NCP_PILOT_3,0.058899
3,BRD-K79131256-001-17-9,BR_NCP_PILOT_3,0.019049
4,Untreated,BR_NCP_PILOT_3,-0.058353


In [5]:
# The grouping we actually want is by group_id (which is `Metadata_line_condition_Metadata_plating_density`)

df[['Metadata_replicate', 'Metadata_group']].groupby(['Metadata_group', 'Metadata_replicate']).size().reset_index(name='counts')

,Metadata_group,Metadata_replicate,counts
0,control_10000,BRD-K15108141-003-04-7,4
1,control_10000,BRD-K21680192-300-14-4,4
2,control_10000,BRD-K36207157-001-09-6,4
3,control_10000,BRD-K79131256-001-17-9,4
4,control_10000,Untreated,32
5,control_2500,BRD-K15108141-003-04-7,4
6,control_2500,BRD-K21680192-300-14-4,4
7,control_2500,BRD-K36207157-001-09-6,4
8,control_2500,BRD-K79131256-001-17-9,4
9,control_2500,Untreated,32


In [6]:
# but grit error!

replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_group",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

AssertionError: grit is calculated for each perturbation independently